# **CSE 7324 Lab 3: Extending Logistic Regression**
### *Thomas Adams, Suleiman Hijazeen, Nancy Le and Andrew Whigham*
------

### **1. Preparation and Overview**
------

### **2. Modeling**
------

### **3. Deployment**
------

### **4. Optimization Using Mean Squared Error**
------

### **5. References**
------